In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import myslack
import datetime as dt
import pickle
import time
import re
from tqdm import trange

In [3]:
driver = webdriver.Chrome()  
url = 'https://www.yogiyo.co.kr/mobile/?utm_source=google&utm_medium=cpc&utm_campaign=sem_bra_orignal&utm_term=%EC%9A%94%EA%B8%B0%EC%9A%94&utm_id=sem_000001&referrer=adjust_tracker%3Dgrm2aa%26adjust_google_network%3Dg%26adjust_google_placement%3D%26adjust_campaign%3Dsem_bra_web_kr_700886066_cpc%26adjust_adgroup%3D38249737818%26adjust_creative%3D%EC%9A%94%EA%B8%B0%EC%9A%94_b&gclid=EAIaIQobChMIv9DaxIL-3AIVA6yWCh0C8glmEAAYASAAEgLbF_D_BwE#/%EC%84%9C%EC%9A%B8/138223/'
driver.get(url) 

In [4]:
food_dict = {
    '프랜차이즈':3, '치킨':4, '피자':5, '양식':5, '중국집':6,
    '한식':7, '일식':8, '돈가스':8, '족발':9, '보쌈':9, 
    '야식':10, '분식':11, '카페':12, '디저트':12 }

In [5]:
# 현재 위치로 설정하기
def set_location():
    driver.find_element_by_xpath('//*[@id="search"]/div/span[1]/button').click()
    print('현재 위치로 설정하는중...')
    time.sleep(5)
    print('현재 위치 설정 완료!')
    
# 카테고리 페이지로 넘어가기
def go_to_category(category):
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(food_dict.get(category))).click()

# 해당 카테고리의 모든 음식점 객체들
def get_all_restaurant_elements():
    return driver.find_elements_by_css_selector('#content > div > div.restaurant-list > div.col-sm-6.contract')
    
# 해당 카테고리의 음식점 페이지로 넘어가기    
def go_to_restaurant(restaurant_num, restaurant_name):
    restaurant_name = driver.find_element_by_class_name('restaurant-name').text
    print(str(restaurant_num)+'번째 음식점 {} 방문'.format(restaurant_name))
    driver.find_element_by_xpath('//*[@id="content"]/div/div[4]/div[{}]/div'.format(restaurant_num)).click()
    time.sleep(2)
    
# 해당 음식점의 리뷰 페이지로 넘어가기
def go_to_review(restaurant_num):
    print(str(restaurant_num)+'번째 음식점 리뷰 페이지로 넘어가는중...')
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(2)
    
# 해당 음식점의 모든 리뷰 객체들
def get_all_review_elements():
    return driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    
# 페이지 한번 맨아래로 내리기
def scroll_bottom():
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

# 카테고리(음식점 리스트) 페이지에서 음식점 리스트 로드하기
def cat_page_scroll_repeat(restaurant_count):
    # 브라우저의 스크롤 위치 조절
    height=200+118*(restaurant_count//2)
    driver.execute_script("window.scrollTo(0,{})".format(height))
    print('카테고리(음식점 리스트) 페이지 (0,{}) 위치로 내리는중...'.format(height))
    time.sleep(3)
    
# 더보기 클릭하기 
def click_more_review():
#     wait = WebDriverWait(driver, 10)
#     element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#review > li.list-group-item.btn-more > a')))
    driver.find_element_by_class_name('btn-more').click()
    time.sleep(2)

# 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in trange(click_count):
        try:
            scroll_bottom()
            print('리뷰 스크롤 내리는중...')
            click_more_review()
        except Exception as e:
            pass
    print('모든 리뷰 불러오기 완료!')
        
# 페이지 뒤로 가기 (한 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감)
def go_back_page():
    driver.execute_script("window.history.go(-1)")    

In [84]:
reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
for review in reviews:
    print('식당이름: '+driver.find_element_by_class_name('restaurant-name').text)    
    print('손님 아이디: '+review.find_element_by_css_selector('span.review-id.ng-binding').text)
    print('메뉴: '+review.find_element_by_css_selector('div.order-items.default.ng-binding').text)
    print('리뷰: '+review.find_element_by_css_selector('p').text)
    print('total score: '+str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))))
    print('taste score: '+review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text)
    print('Quantity score: '+review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text)
    print('Delivery score: '+review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text)
    print('Date: '+review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text)
    print()

식당이름: 처갓집양념치킨-구의자양점
손님 아이디: po**님
메뉴: 반반치킨/1
리뷰: 양념은 양념대로 후라이드는 후라이드대로 맛있게 먹었습니다
total score: 5
taste score: 5
Quantity score: 5
Delivery score: 5
Date: 14시간 전

식당이름: 처갓집양념치킨-구의자양점
손님 아이디: mi**님
메뉴: 슈프림양념치킨/1(추가 선택(순살로 변경))
리뷰: ㅅㅅㅅㅅㅅㅅㅅㅅㅅㅅ
total score: 5
taste score: 5
Quantity score: 5
Delivery score: 5
Date: 21시간 전

식당이름: 처갓집양념치킨-구의자양점
손님 아이디: kj**님
메뉴: 반반치킨/1(추가 선택(순살로 변경))
리뷰: 맛,양 괜찮았습니다. 배달 빠르고 괜찮았고요~
total score: 5
taste score: 5
Quantity score: 5
Delivery score: 4
Date: 어제

식당이름: 처갓집양념치킨-구의자양점
손님 아이디: nh**님
메뉴: 슈프림양념치킨/1(추가 선택(순살로 변경))
리뷰: 맛은 단거 좋아하시는분 드시면되고 양은 좀 작네요 배달은 1시간 걸렸습니다 8.20일 월 10시 40분에 시켰는데
total score: 5
taste score: 3
Quantity score: 3
Delivery score: 3
Date: 어제

식당이름: 처갓집양념치킨-구의자양점
손님 아이디: au**님
메뉴: 올리고당양념/1(추가 선택(순살로 변경))
리뷰: 달다 맛있다 하지만 약간물린다
total score: 4
taste score: 4
Quantity score: 4
Delivery score: 4
Date: 2일 전

식당이름: 처갓집양념치킨-구의자양점
손님 아이디: au**님
메뉴: 치폴레양념치킨/1
리뷰: 은근히 맵다 엽떡 보통맛약간아래인듯 다음날 배아프다조심
total score: 4
taste score: 4
Quantity score: 4
Delivery score: 

In [90]:
restaurants =  driver.find_elements_by_css_selector('#content > div > div.restaurant-list > div.col-sm-6.contract')

In [ ]:
def yogiyo_crawling(category):
    try:
        set_location()
        go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감 
        print(category+'페이지로 넘어가는 중...')
        time.sleep(2)
        print(category+'페이지 로드 완료!')
        
        df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                                   'Total','Taste','Quantity','Delivery','Date'])

        print('Start {} Crawling...'.format(category))
        for i in trange(get_all_restaurant_count()): # 해당 카테고리의 음식점 개수만큼 loop
            try:
                cat_page_scroll_repeat(i) # i번째 음식점 위치로 page를 내림
                go_to_restaurant(i+2) # i번째 음식점 페이지로 넘어감
                go_to_review(i+2) # 해당 음식점의 리뷰페이지로 넘어감
                stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴
                
                # 불러온 모든 리뷰를 객체로 받아옴
                reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')

                for review in get_all_review_elements():  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
                    try:
                        df.loc[len(df)] = { 
                            'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                            'UserID':review.find_element_by_css_selector('span.review-id.ng-binding').text,
                            'Menu':review.find_element_by_css_selector('div.order-items.default.ng-binding').text,
                            'Review':review.find_element_by_css_selector('p').text,
                            'Total':str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                            'Taste':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                            'Quantity':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                            'Delivery':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text,
                            'Date':review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text,
                        }
                    except Exception as e:
                        print('리뷰 페이지 에러')
                        print(e)
                        pass

                go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
                print('음식점 리스트 페이지로 돌아가는중...')
                time.sleep(2)
            except Exception as e:
                print('음식점 리스트 페이지 에러')
                print(e)
                pass
            
    except Exception as e:
        pass
    finally:
        print('Finish {} Crawling!!!'.format(category))
        return df

In [ ]:
%time chinese_df = yogiyo_crawling('중국집')

In [ ]:
chicken_df.tail()

In [ ]:
myslack.send_slack('finish!')

In [ ]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기

---

In [ ]:
# def yogiyo_crawling(category):
#     go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감 
#     print(category+'페이지로 넘어가는 중...')
#     time.sleep(2)
    
#     df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
#                                'Taste','Quantity','Delivery','Date'])
    
#     print('Start {} Crawling...'.format(category))
#     for i in trange(get_store_count()): # 해당 카테고리의 음식점 개수만큼 돌아감
#         try:
#             scroll_bottom()
#             time.sleep(2)
#             go_to_store(i+2) # 순서대로 각 음식점 페이지로 넘어감
#             print(str(i+1)+'번째 음식점 페이지로 넘어가는중...')
#             time.sleep(2)
#             go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
#             print('해당 음식점 리뷰 페이지로 넘어가는중...')
#             time.sleep(2)
#             stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴
            
#             for j in trange(get_review_count()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
#                 try:
#                     df.loc[len(df)] = { 
#                         'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
#                         'UserID':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[1]/span[1]'.format(j+2)).text,
#                         'Menu':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[3]'.format(j+2)).text,
#                         'Review':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/p'.format(j+2)).text,
#                         'Taste':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[3]'.format(j+2)).text,
#                         'Quantity':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[6]'.format(j+2)).text,
#                         'Delivery':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[9]'.format(j+2)).text,
#                         'Date':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[1]/span[2]'.format(j+2)).text,
#                     }
#                 except Exception as e:
#                     print('리뷰 페이지 에러')
#                     print(e)
#                     pass
                
#             go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
#             print('음식점 리스트 페이지로 돌아가는중...')
#             time.sleep(2)
#             scroll_bottom()
#             time.sleep(2)
            
#         except Exception as e:
#             print('음식점 페이지 에러')
#             scroll_bottom()
#             print(e)
#             pass
                
#     print('Finish {} Crawling!!!'.format(category))
#     return df